# TODO
* Tokenize text
* Investigate more model

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [31]:
import sys
import gc

import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
gpus = tf.config.list_physical_devices("GPU")
print(gpus)
tf.config.set_logical_device_configuration(
    gpus[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=8000)],
)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-04-22 17:05:35.416192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 17:05:35.429579: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-22 17:05:35.429708: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
text = open("./data/robert_frost.txt").read().lower()

In [5]:
tokenizer = Tokenizer(num_words=3000, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t', lower=True, oov_token="<UNK>")
tokenizer.fit_on_texts([text])
sequences = tokenizer.texts_to_sequences([text])
padded_sequences = pad_sequences(sequences, maxlen=55, padding='post', truncating='post')

In [15]:
len(np.unique(sequences[0]))

2999

In [16]:
tokenizer.sequences_to_texts([sequences[0][:20]])

["the pasture\n\n\ni'm going out to clean the pasture spring \ni'll only stop to rake the leaves away\n and wait to"]

In [17]:
max_length = 8
step = 5

sentence = []
next_char = []

for i in range(0, len(sequences[0]) - max_length, step):
    sentence.append(sequences[0][i : i + max_length])
    next_char.append(sequences[0][i + max_length])


X, Y = np.array(sentence), keras.utils.to_categorical(next_char)

In [18]:
X.shape

(4275, 8)

In [66]:
print(gc.collect())
K.clear_session()

model = Sequential()

layers = [
    Embedding(tokenizer.num_words, 1024),
    # Bidirectional(LSTM(2048, return_sequences=True)),
    # Bidirectional(LSTM(2048, return_sequences=True)),
    # Bidirectional(LSTM(1024)),
    LSTM(1024),
    Dense(tokenizer.num_words, activation="softmax"),
]

model = tf.keras.Sequential(layers)
model.compile(loss="categorical_crossentropy", optimizer="adam")

38017


In [67]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 1024)        3072000   
                                                                 
 lstm (LSTM)                 (None, 1024)              8392704   
                                                                 
 dense (Dense)               (None, 3000)              3075000   
                                                                 
Total params: 14539704 (55.46 MB)
Trainable params: 14539704 (55.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [68]:
es = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=5,
    restore_best_weights=True
)

In [69]:
model.fit(X, Y, validation_split=0.2, epochs=20, callbacks=[es])

Epoch 1/20
107/107 [==============================] - 5s 29ms/step - loss: 6.8738 - val_loss: 6.1040
Epoch 2/20
107/107 [==============================] - 1s 8ms/step - loss: 6.0214 - val_loss: 6.1395
Epoch 3/20
107/107 [==============================] - 1s 9ms/step - loss: 5.7098 - val_loss: 6.2184
Epoch 4/20
107/107 [==============================] - 1s 8ms/step - loss: 5.2503 - val_loss: 6.5527
Epoch 5/20
107/107 [==============================] - 1s 9ms/step - loss: 4.5700 - val_loss: 6.8847
Epoch 6/20
107/107 [==============================] - 1s 10ms/step - loss: 3.6449 - val_loss: 7.3382


In [70]:
# model.save("./models/model.keras")

In [80]:
X[:10]

array([[   2, 1456,  109,   26,    3,  931,    2,  232],
       [ 931,    2,  232,  394,  233,  114,  395,    3],
       [ 114,  395,    3,  932,    2,  209, 1457,    6],
       [ 209, 1457,    6,  449,    3,  450,    2,  194],
       [ 450,    2,  194,  504,    4,  132,   27,  396],
       [ 132,   27,  396,   29,  123,  124,   79,    7],
       [ 124,   79,    7,   59,   95, 1458,  109,   26],
       [1458,  109,   26,    3,  701,    2,   96, 1459],
       [   2,   96, 1459,  451,   64,    2,  397,   62],
       [   2,  397,   62,   38,  254,  125, 1460,   56]])

In [159]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas), preds

def genText(seq, tokenizer: Tokenizer, number_gen=200, temp=1):
    generated = seq
    sent = seq
    text_in = tokenizer.sequences_to_texts(seq)
    # print(text_in, end='')
    
    for _ in range(number_gen):
        print(sent[0])
        t = list()
        
        pred = model.predict(sent, verbose=0)[0]
        # next_index, _  = sample(pred, temp)
        next_index = np.argmax(pred)
        next_char = tokenizer.sequences_to_texts([[next_index]])
        
        generated += [next_index]
        sent = sent[0][1:] + [next_index]
        t.append(sent)
        t = np.array(t)
        sent = t.copy()
        
        sys.stdout.write(next_char[0])
        sys.stdout.flush()

In [160]:
text2gen = [sequences[0][2000:2008]]
genText(text2gen, tokenizer, 200)

[89, 99, 3, 34, 9, 8, 305, 12]
the[ 99   3  34   9   8 305  12   2]
the[  5  36  11  10 307  14   4]
the[ 38  13  12 309  16   6]
the[ 15  14 311  18   8]
the[ 16 313  20  10]
the[315  22  12]
the[24 14]
the[16]
the[]


2024-04-22 18:02:35.253223: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at cudnn_rnn_ops.cc:1771 : INVALID_ARGUMENT: max_seq_length <= 0


InvalidArgumentError: Graph execution error:

Detected at node CudnnRNN defined at (most recent call last):
<stack traces unavailable>
max_seq_length <= 0
	 [[{{node CudnnRNN}}]]
	 [[sequential_1/lstm/PartitionedCall]] [Op:__inference_predict_function_58807]